#  session extraction

In [ ]:
import json

# function to turn string returned from the LLM into valid python dictionary
def extract_json(text):
    def find_closing(text, open_pos, open_char, close_char):
        balance = 0
        for i in range(open_pos, len(text)):
            if text[i] == open_char:
                balance += 1
            elif text[i] == close_char:
                balance -= 1
                if balance == 0:
                    return i
        return -1

    obj_start = text.find("{")
    arr_start = text.find("[")

    if obj_start == -1 and arr_start == -1:
        return {}, None  

    start_index = obj_start if arr_start == -1 or (obj_start != -1 and obj_start < arr_start) else arr_start
    open_char = "{" if start_index == obj_start else "["
    close_char = "}" if open_char == "{" else "]"

    end_index = find_closing(text, start_index, open_char, close_char)

    if start_index != -1 and end_index != -1:
        json_text = text[start_index:end_index + 1]
        try:
            json_response = json.loads(json_text)
            return json_response, text[end_index + 1:]
        except json.JSONDecodeError:
            return {}, None  
    else:
        return {}, None

In [ ]:
summary_prompt = """
    Analyze the provided list of Google search records to identify distinct topic groups. For each group, create a summary in the JSON format below. Ensure each summary includes: 

    - `time_start`: The start time of the first search in the group.
    - `time_end`: The end time of the last search in the group.
    - `description`: A detailed account of the searches and site visits, enriched with inferred user intent and additional insights into the topic.

    Each `description` should not only recap the searches but also offer a deeper understanding of what the user might be seeking or the broader context of their inquiries. Group searches based on thematic relevance and timing. 

    Example of JSON output format:

    {
    "time_start": "HH:MM",
    "time_end": "HH:MM",
    "description": "Elaborate on what the user did and why, based on the search terms and visited pages.",
    }
    
    Here is a list of searches:
"""
chunk_size = 15


for filename in tqdm(get_filenames()):

    # We send chunks of 15 raw search records to the LLM to get more accurate results
    for i in tqdm(range(0, len(df), chunk_size)):
        chunk = df.iloc[i : i + chunk_size]

        answer = get_completion(f"{summary_prompt}\n{chunk}")

        # Sometimes the LLM returns multipe json objects in a list
        # Some other times it returns a single json object
        # We need to handle both cases
        parsed_results = []
        while answer:
            parsed_result, answer = extract_json(answer)

            if parsed_result:
                if isinstance(parsed_result, dict):
                    parsed_results.append(parsed_result)
                elif isinstance(parsed_result, list):
                    parsed_results.extend(parsed_result)

        parsed_df = pd.concat(
            [parsed_df, pd.DataFrame(parsed_results)], ignore_index=True
        )


    parsed_df.to_csv(os.path.join(out_path, f"{date}.csv"))



# embeddings

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("Salesforce/SFR-Embedding-Mistral")
embeddings = model.encode(parsed_df["description"])

# building the DAG

In [ ]:
from pgvector.psycopg import register_vector
import psycopg
import os

conn = psycopg.connect(**psycopg.conninfo.conninfo_to_dict(os.environ["DATABASE_URL"]))
conn.autocommit = True

conn.execute("CREATE EXTENSION IF NOT EXISTS vector")
register_vector(conn)

conn.execute("DROP TABLE IF EXISTS documents")
conn.execute(
    """CREATE TABLE documents (
             id bigserial PRIMARY KEY, 
             description text,
             date DATE, 
             time TIME,
             raw text,
             embedding vector(1536) 
    )"""
)

# TODO populate the documents table with the dataframe contents

conn.execute("DROP TABLE IF EXISTS edges")
conn.execute(
    """CREATE TABLE edges (
             id bigserial PRIMARY KEY, 
             parent_id bigint,
             child_id bigint,
             weight float
    )"""
)

# Code to merge sessions that are the same (this can happen because of the chunking we do in step 1)
# for example: session 1 starts at 10:15 and ends at 10:30, session 2 starts at 10:30 and ends at 10:45 and they have the same description

# Calculate the 10th percentile for time intervals (in seconds)
time_threshold = conn.execute("""
WITH LaggedDocuments AS (
    SELECT
        date,
        time,
        LAG(time) OVER (ORDER BY date, time) AS prev_time
    FROM
        documents
    WHERE
        is_taxonomy = FALSE
),
TimeDifferences AS (
    SELECT
        EXTRACT(EPOCH FROM (time - prev_time)) AS time_diff
    FROM
        LaggedDocuments
    WHERE
        time > prev_time
)
SELECT
    percentile_cont(0.10) WITHIN GROUP (ORDER BY time_diff) AS time_interval_10th
FROM
    TimeDifferences;

""").fetchone()[0]

# Calculate the 90th percentile for embedding similarities using cosine similarity
embedding_similarity_threshold = conn.execute("""
    WITH CosineSimilarities AS (
        SELECT
            date,
            time,
            1 - (embedding <=> LAG(embedding) OVER (ORDER BY date, time)) AS cosine_similarity
        FROM
            documents
        WHERE
            is_taxonomy = FALSE
    ),
    FilteredSimilarities AS (
        SELECT
            cosine_similarity
        FROM
            CosineSimilarities
        WHERE
            cosine_similarity IS NOT NULL
    )
    SELECT
        percentile_cont(0.90) WITHIN GROUP (ORDER BY cosine_similarity) AS embedding_similarity_90th
    FROM
        FilteredSimilarities;
""").fetchone()[0]

print(f"Time threshold: {time_threshold} seconds. Embedding similarity threshold: {embedding_similarity_threshold}")

# Time threshold: 60.0 seconds. Embedding similarity threshold: 0.8296323921172489
# this means that we will merge documents that are within 60 seconds of each other and have an embedding similarity of 0.83 or higher

# merge similar documents within the time and embedding similarity thresholds
records = conn.execute(
    """
SELECT a.id, b.id, (1 - (a.embedding <=> b.embedding)) AS similarity
FROM documents a
JOIN documents b ON a.id < b.id
WHERE ABS(EXTRACT(EPOCH FROM (
    (a.date || ' ' || a.time)::timestamp - 
    (b.date || ' ' || b.time)::timestamp))/60) <= %s
    AND NOT (a.is_taxonomy = TRUE OR b.is_taxonomy = TRUE )
""",
    ((time_threshold / 60),),
).fetchall()


candidates_to_merge = []
for record in records:
    doc_id_a, doc_id_b, similarity = record
    if similarity >= embedding_similarity_threshold:
        candidates_to_merge.append((doc_id_a, doc_id_b))

# function to create the dag 
# for every document, we find the most similar document that cmae before in time and over a certain similarity threshold (0.6)
# we create an edge between the two documents with a weight of 1 - similarity (the distance)

def create_dag():
    return conn.execute(
        """
    WITH DocumentPairs AS (
        SELECT
            a.id AS doc_id,
            b.id AS compared_doc_id,
            b.is_taxonomy AS compared_is_taxonomy,
            (1 - (a.embedding <=> b.embedding)) AS similarity,
            a.date AS doc_date,
            a.time AS doc_time,
            b.date AS compared_doc_date,
            b.time AS compared_doc_time
        FROM
            documents a
        JOIN
            documents b ON a.id != b.id AND 
                        (a.date > b.date OR (a.date = b.date AND a.time > b.time))
    ),
    RankedPairs AS (
        SELECT
            *,
            ROW_NUMBER() OVER(PARTITION BY doc_id, compared_is_taxonomy ORDER BY similarity DESC) AS rank
        FROM
            DocumentPairs
    ), FilteredPairs1 as (
        SELECT
            doc_id,
            compared_doc_id,
            compared_is_taxonomy,
            similarity
        FROM
            RankedPairs
        WHERE
            rank = 1 AND
            ((compared_is_taxonomy = FALSE AND similarity > 0.6) OR compared_is_taxonomy = TRUE)
    ), FilteredPairs2 as (
        SELECT
            doc_id,
            MAX(compared_doc_id) AS compared_doc_id,
            MAX(similarity) AS similarity
        FROM
            FilteredPairs1
        group by doc_id
    )
    INSERT INTO edges (parent_id, child_id, weight)
    SELECT
        doc_id,
        compared_doc_id,
        1-similarity
    FROM
        FilteredPairs2;
""")


# send results to GPT4

In [ ]:
TODO